In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Настройки Pandas
pd.set_option('display.max_columns', None)

In [2]:
# Чтение датасета
df = pd.read_csv('../../data/_data.csv')

In [3]:
# Удаляем столбец `Unnamed: 0`
df = df.iloc[:, 1:]

# Переименовываем столбцы
NEW_COLUMNS = {
    'ID  объявления':           'id',
    'Количество комнат':        'rooms',
    'Тип':                      'type',
    'Метро':                    'metro_station',
    'Адрес':                    'address',
    'Площадь, м2':              'area',
    'Дом':                      'building_type',
    'Парковка':                 'parking',
    'Цена':                     'price',
    'Телефоны':                 'phones',
    'Описание':                 'description',
    'Ремонт':                   'renovation',
    'Площадь комнат, м2':       'rooms_area',
    'Балкон':                   'balcony',
    'Окна':                     'windows',
    'Санузел':                  'bathroom',
    'Можно с детьми/животными': 'kids_pets',
    'Дополнительно':            'extra',
    'Название ЖК':              'complex_title',
    'Серия дома':               'building_series',
    'Высота потолков, м':       'ceiling_height',
    'Лифт':                     'elevator',
    'Мусоропровод':             'trash_chute',
    'Ссылка на объявление':     'listing_url',
}
df = df.rename(columns=NEW_COLUMNS)

### Колонка `rooms`

In [20]:
# Создаем столбец `num_rooms` с кол-ом комнат
df['num_rooms'] = df['rooms'].str.extract(r'(\d+)').astype('Int64')
# Создаем столбец `type_rooms` с типом комнат 'изолированная', ' смежная'
df['type_rooms'] = df['rooms'].str.extract(r'(\s\D+)')

### Колонка `metro_station`

In [5]:
# Выделим из колонки `metro_station` сведения о времени до метро
pattern = r"\((\d+)\s*мин"
minutes_str = df['metro_station'].str.extract(pattern, expand=False)
df['time_to_metro'] = pd.to_numeric(minutes_str, errors="coerce")

### Колонка `area`

In [6]:
# Добавляем столбец с общей площадью квартир
df['total area'] = df['area'].str.split('/').str[0].astype('float')

### Колонка `building_type`

In [7]:
# Добавляем столбец `floor` с номером этажа и `total_floors` с этажностью здания
df['floor'] = df['building_type'].str.extract(r'(\d{1,3})/').astype('Int64')
df['total_floors'] = df['building_type'].str.extract(r'/(\d{1,3})').astype('Int64')
df['parking'].unique()

array(['подземная', nan, 'наземная', 'открытая', 'многоуровневая',
       'на крыше'], dtype=object)

### Колонка `parking`

In [8]:
# категоризовываем колонку `parking` в колонку `parking_categories` от 0 до 5
df['parking'].fillna('нет', inplace=True)
df['parking_categories'] = pd.factorize(df['parking'])[0]

### Столбец `Price`

In [9]:
# Создаем столбец с главной ценой
# r'([\d\.]+)' Это регулярное выражение, которое используется для поиска чисел с точкой. 
# \d — означает любую цифру от 0 до 9. 
# [\d\.]+ — это: «одна или больше цифр и/или точек подряд». 
df['main_price'] = df['price'].str.extract(r'([\d\.]+)').astype(float)

# Поработаем с залогом
# Ищем слово Залог, дефис, и число после него
# *-\s* это мы так страхуемся, чтобы захватились пробелы до и после дефиса"""
df['deposit'] = df['price'].str.extract(r'Залог\s*-\s*([\d\s]+)')[0]
df['deposit'] = df['deposit'].str.replace(' ', '').astype(float)

# Коммунальные услуги - выводим 1 если True и 0 если False
df['utilities_included'] = df['price'].str.contains('Коммунальные услуги включены', case=False, na=False).astype(int)


# Срок аренды
# Ищем слово Срок аренды, дефис, и слова после него
df['rental_term'] = df['price'].str.extract(r'Срок аренды\s*-\s*([а-яА-Яa-zA-Z ]+)')

# Предоплата
# Ищем слово Предоплата, дефис, и слова после него
df['prepayment_months'] = df['price'].str.extract(r'Предоплата\s*(\d+)')
df['prepayment_months'] = df['prepayment_months'].astype(float).astype('Int64') 
# делаем через  astype('Int64') чтобы не было проблем с пропусками
# так как обычный int не поддерживает NaN — это тип float.

### Столбец `balcony`

In [10]:
# Добавляем столбец `presence_balcony` с информацией 0 - балконов/лоджий нет, 1 - балкон/лоджия есть
def extract_numbers(balcony):
    if isinstance(balcony, str): # проверяем на отсутвтвие Nun 
        return int(1)
    else:
        return int(0)
df['presence_balcony'] = df['balcony'].apply(extract_numbers)

### Столбец `bathroom`

In [11]:
# Добавляем столбец `numbers_toilets` с информацией о кол-ве санузлов
def numbers_toilet(bathroom):
    if isinstance(bathroom, str): #проверяем на отсутвтвие Nun 
        numbers = re.findall(r'\d+', bathroom)
        return sum(int(num) for num in numbers)
    else:
        return int(0)
df['numbers_toilets'] = df['bathroom'].apply(numbers_toilet)

### Столбец `kids_pets`

In [12]:
# Добавляем столбец `kids_allowed` с информацией допустима ли аренда с детьми
df['kids_allowed'] = np.where(df['kids_pets'].str.contains('детьми'), 1, 0)
# Добавляем столбец `pets_allowed` с информацией допустима ли аренда с животными
df['pets_allowed'] = np.where(df['kids_pets'].str.contains('животными'), 1, 0)

### Столбец `extra`

In [13]:
# добавляем столбец с наличием/отсутствием мебели
df["furniture"] = np.where(df["extra"].isna(), np.nan, df["extra"].str.contains("Мебель в комнатах", na=False).astype(int))
# добавляем столбец с наличием/отсутствием холодильника
df["fridge"] = np.where(df["extra"].isna(), np.nan, df["extra"].str.contains("Холодильник", na=False).astype(int))
# добавляем столбец с наличием/отсутствием стиральной машины
df["washing_machine"] = np.where(df["extra"].isna(), np.nan, df["extra"].str.contains("Стиральная машина", na=False).astype(int))

### Столбец `ceiling_height`

In [14]:
# Заполняем пропущенные значения в столбце `ceiling_height` медианой
df.fillna({'ceiling_height': (df['ceiling_height'].median())}, inplace=True)

### Столбец `elevator`

In [15]:
# Вытащим из строк информацию о количестве лифтов
df['elevator_num'] = df['elevator'].str.extract(r'(\d{1,2})\).*(\d{1,2})\).*|(\d{1,2})\)').astype('Int64').sum(axis=1, min_count=1)

### Столбец `trash_chute`

In [16]:
# 0 - мусоропровода нет, 1 - мусоропровод имеется
df['trash_chute'] = df['trash_chute'].map({'Да': 1, 'Нет': 0}).astype('Int64')

In [17]:
df['num_rooms'] = df['rooms'].str.extract(r'(\d+)')
df['num_rooms'] = df['num_rooms'].fillna(df['num_rooms'].dropna().astype('int64').median()).astype('int64')
df['num_rooms'].unique()

array([4, 2, 3, 5, 6, 1])

In [18]:
df['type_rooms'] = df['rooms'].str.extract(r'(\s\D+)')
df['type_rooms'].unique()


array([nan, ' Оба варианта', ' Изолированная', ' Смежная'], dtype=object)

In [21]:
df

,id,rooms,type,metro_station,address,area,building_type,parking,price,phones,description,renovation,rooms_area,balcony,windows,bathroom,kids_pets,extra,complex_title,building_series,ceiling_height,elevator,trash_chute,listing_url,num_rooms,type_rooms,time_to_metro,total area,floor,total_floors,parking_categories,main_price,deposit,utilities_included,rental_term,prepayment_months,presence_balcony,numbers_toilets,kids_allowed,pets_allowed,furniture,fridge,washing_machine,elevator_num
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.00,"Пасс (4), Груз (1)",1,https://www.cian.ru/rent/flat/271271157,4,NaN,9.0,200.0,5,16,0,500000.0,500000.0,1,Длительный,1,0,0,1,1,1.0,1.0,1.0,5
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.50,"Пасс (1), Груз (1)",0,https://www.cian.ru/rent/flat/271634126,4,NaN,8.0,198.0,5,16,0,500000.0,500000.0,1,Длительный,1,0,3,1,0,1.0,1.0,1.0,2
2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.20,Пасс (1),<NA>,https://www.cian.ru/rent/flat/271173086,4,Оба варианта,7.0,200.0,5,16,0,500000.0,500000.0,0,Длительный,1,0,3,1,0,1.0,1.0,1.0,1
3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.20,Пасс (1),<NA>,https://www.cian.ru/rent/flat/272197456,4,Оба варианта,3.0,170.0,5,6,0,400000.0,400000.0,0,Длительный,1,0,3,0,1,1.0,1.0,1.0,1
4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",нет,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.90,"Пасс (1), Груз (1)",1,https://www.cian.ru/rent/flat/273614615,2,NaN,7.0,58.0,12,26,1,225000.0,225000.0,1,Длительный,1,0,2,1,1,1.0,1.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,NaN,Квартира,м. Говорово (8 мин пешком),"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,нет,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",+79154383401,Сдаётся светлая теплая квартира (студия) с бал...,Евроремонт,NaN,Балкон (1),NaN,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Мещерский лес,NaN,2.64,"Пасс (1), Груз (1)",<NA>,https://www.cian.ru/rent/flat/215565511,<NA>,NaN,8.0,35.0,10,14,1,42000.0,42000.0,1,Длительный,1,1,1,1,1,1.0,1.0,1.0,2
23364,274654844,1,Квартира,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитны